In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from afinn import Afinn
afinn=Afinn(emoticons=True)

In [2]:
df=pd.read_excel('Data_reviews1_to_44_Final.xlsx')
df['Review Date']=pd.to_datetime(df['Review Date'])
df=df.sort_values('Review Date',ascending=False).reset_index()
df=df.drop('index',axis=1)
df.to_csv('forR.csv',sep='|')

In [28]:
files=sorted(os.listdir('/Users/FrankLIUChangxuan/Desktop/Proj/quarter_rev'))
files

['Year2016_Q1.csv',
 'Year2016_Q2.csv',
 'Year2016_Q3.csv',
 'Year2016_Q4.csv',
 'Year2017_Q1.csv',
 'Year2017_Q2.csv',
 'Year2017_Q3.csv',
 'Year2017_Q4.csv',
 'Year2018_Q1.csv',
 'Year2018_Q2.csv',
 'Year2018_Q3.csv',
 'Year2018_Q4.csv']

In [49]:
##Overall topics
def getTopicSentiment(file):
    
    d1=pd.read_csv('quarter_rev/'+file,index_col=0)
    vectorizer1=CountVectorizer(ngram_range=(1,1),min_df=1,stop_words='english')
    X=vectorizer1.fit_transform(d1['Review'])
    terms=vectorizer1.get_feature_names()
    lda=LatentDirichletAllocation(n_components=5,learning_method='online').fit(X)
    topics=[' '.join([terms[i] for i in topic.argsort()[:-5-1:-1]]) for topic_idx,topic in enumerate(lda.components_)]
    mtx=[lda.fit_transform(X[k]) for k in range(X.shape[0])]
    print(topics)

    t0=[mtx[i][0][0] for i in range(len(mtx))]
    t1=[mtx[i][0][1] for i in range(len(mtx))]
    t2=[mtx[i][0][2] for i in range(len(mtx))]
    t3=[mtx[i][0][3] for i in range(len(mtx))]
    t4=[mtx[i][0][4] for i in range(len(mtx))]
    #t5=[mtx[i][0][4] for i in range(len(mtx))]
    #t6=[mtx[i][0][4] for i in range(len(mtx))]
    #t7=[mtx[i][0][4] for i in range(len(mtx))]
    #t8=[mtx[i][0][4] for i in range(len(mtx))]
    #t9=[mtx[i][0][4] for i in range(len(mtx))]
    
    d1=pd.concat([d1,pd.DataFrame({'topic0':t0,'topic1':t1,'topic2':t2,'topic3':t3,'topic4':t4})],axis=1)
    d1=d1.sort_values('Restaurant.name').drop([0])
    
    #sentiment
    r_names=[]
    shr=[]
    time=[]
    score_t0=[]
    score_t1=[]
    score_t2=[]
    score_t3=[]
    score_t4=[]
    #score_t5=[]
    #score_t6=[]
    #score_t7=[]
    #score_t8=[]
    #score_t9=[]
    rating=[]
    for r_name in d1['Restaurant.name'].unique():
        r_names.append(r_name)
        time.append(int(file[4:8]+'0'+file[10]))
        r=d1.loc[d1['Restaurant.name']==r_name]
        shr.append(len(r.index)/len(d1.index))
        score_t0.append(afinn.score(''.join([i for i in r.loc[r['topic0']>0.8]['Review']])))
        score_t1.append(afinn.score(''.join([i for i in r.loc[r['topic1']>0.8]['Review']])))
        score_t2.append(afinn.score(''.join([i for i in r.loc[r['topic2']>0.8]['Review']])))
        score_t3.append(afinn.score(''.join([i for i in r.loc[r['topic3']>0.8]['Review']])))
        score_t4.append(afinn.score(''.join([i for i in r.loc[r['topic4']>0.8]['Review']])))
        #score_t5.append(afinn.score(''.join([i for i in r.loc[r['topic5']>0.8]['Review']])))
        #score_t6.append(afinn.score(''.join([i for i in r.loc[r['topic6']>0.8]['Review']])))
        #score_t7.append(afinn.score(''.join([i for i in r.loc[r['topic7']>0.8]['Review']])))
        #score_t8.append(afinn.score(''.join([i for i in r.loc[r['topic8']>0.8]['Review']])))
        #score_t9.append(afinn.score(''.join([i for i in r.loc[r['topic9']>0.8]['Review']])))
        rating.append(np.mean([i for i in r['Rating']]))
    return pd.DataFrame({'mkt_shr':shr,
                         'Restaurant.name':r_names,
                         'Review.date':time,
                         'score_t0':score_t0,'score_t1':score_t1,'score_t2':score_t2,'score_t3':score_t3,'score_t4':score_t4,
                         'Rating':rating})
                         #'score_t5':score_t5,'score_t6':score_t6,'score_t7':score_t7,'score_t8':score_t8,'score_t9':score_t9})

In [50]:
dfs=[getTopicSentiment(x) for x in files]

['food good great place salsa', 'tacos taco food place mexican', 'tacoma approached crusted servers sorry', 'good really food salsa place', 'enchilada breakfast dairy taqueria manager']
['food good tacos place mexican', 'got good food happy bar', 'password man cash soup counter', 'radishes music wallet barrio yes', 'happening bad food just beach']
['tacos taco meat asada like', 'food good place great tacos', 'later source details toilet unbelievable', 'pregnant lady tops boys asking', 'lo coffee scene believe dude']
['tacos food place good asada', 'price lunch hole portion extremely', 'delivery consistent bitesquad nogada chiles', 'food table service good just', 'food great good place mexican']
['gluten free given beef er', 'food place good tacos great', 'item sopes substance substantial recommendation', 'hopes lemon delivery menus wearing', 'blah business closing open food']
['downstairs flatbread pharmacy chicharrones connected', 'mezcal flan ordered cactus server', 'jaime business n

In [51]:
all_names=[df['Restaurant.name'] for df in dfs]
rst_always=set.intersection(*map(set,all_names))
dfs=[df.loc[df['Restaurant.name'].isin(rst_always)] for df in dfs]
final=pd.concat(dfs)
final.to_csv('final.csv',index=False)